In [1]:
import tensorflow as tf
import numpy as np

/home/hojoon/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/hojoon/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../data/mnist/", one_hot = True)

Extracting ../data/mnist/train-images-idx3-ubyte.gz
Extracting ../data/mnist/train-labels-idx1-ubyte.gz
Extracting ../data/mnist/t10k-images-idx3-ubyte.gz
Extracting ../data/mnist/t10k-labels-idx1-ubyte.gz


### Build Graph

In [3]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

#keep_prob = tf.placeholder(tf.float32)
is_train = tf.placeholder(tf.bool, name='is_train')

In [4]:
# Width, Height, Depth, KernalSize
W1 = tf.Variable(tf.random_normal([3,3,1,32],stddev = 0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1,1,1,1], padding = 'SAME')
L1 = tf.layers.batch_normalization(L1, center=True, scale=True,training=is_train)
L1 = tf.nn.relu(L1)
A2 = tf.nn.max_pool(L1, ksize = [1,2,2,1], strides=[1,2,2,1], padding = 'SAME')

print(X)
print(L1)
print(A2)

Tensor("Placeholder:0", shape=(?, 28, 28, 1), dtype=float32)
Tensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)


In [5]:
W2 = tf.Variable(tf.random_normal([3,3,32,64],stddev = 0.01))
L2 = tf.nn.conv2d(A2, W2, strides=[1,1,1,1], padding = 'SAME')
L2 = tf.layers.batch_normalization(L2, center=True, scale=True,training=is_train)
L2 = tf.nn.relu(L2)
A3 = tf.nn.max_pool(L2, ksize = [1,2,2,1], strides=[1,2,2,1], padding = 'SAME')

print(A2)
print(L2)
print(A3)

Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)
Tensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)


In [6]:
# Width, Height, Depth, KernalSize
W3 = tf.Variable(tf.random_normal([7*7*64, 256],stddev = 0.01))
A3 = tf.reshape(A3, [-1, 7*7*64])
L3 = tf.matmul(A3, W3)
A4 = tf.nn.relu(L3)

print(A3)
print(L3)
print(A4)

Tensor("Reshape:0", shape=(?, 3136), dtype=float32)
Tensor("MatMul:0", shape=(?, 256), dtype=float32)
Tensor("Relu_2:0", shape=(?, 256), dtype=float32)


In [7]:
# Width, Height, Depth, KernalSize
W4 = tf.Variable(tf.random_normal([256, 10],stddev = 0.01))
score = tf.matmul(A4, W4)

print(A4)
print(score)

Tensor("Relu_2:0", shape=(?, 256), dtype=float32)
Tensor("MatMul_1:0", shape=(?, 10), dtype=float32)


In [8]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = score, labels = Y))
optimizer = tf.train.AdamOptimizer(1e-3)

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_op = optimizer.minimize(loss)

In [9]:
is_correct = tf.equal(tf.argmax(score, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.to_float(is_correct))

In [10]:
# To track in the tensorboard
tf.summary.scalar('loss',loss)
tf.summary.scalar('accuracy',accuracy)

merged = tf.summary.merge_all()

### Train Graph

In [11]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Allocate the directory to save the graph
    test_writer = tf.summary.FileWriter('./logs/test', sess.graph)
    
    batch_size  = 128
    total_batch = mnist.train.num_examples // batch_size
    
    for epoch in range(30):
        total_loss = 0
        
        for _ in range(total_batch):
            x_batch, y_batch = mnist.train.next_batch(batch_size)
            x_batch = x_batch.reshape(-1,28,28,1)
            
            _, loss_val = sess.run([train_op, loss], feed_dict={X:x_batch, Y:y_batch, is_train:True})
            total_loss += loss_val
            
        print('Epoch:','%04d' %(epoch+1),
              'Avg Loss: %.3f'%(total_loss / total_batch))
       
        summary_test = sess.run(merged, feed_dict = {X:mnist.test.images.reshape(-1,28,28,1), Y:mnist.test.labels, is_train:False})
        test_writer.add_summary(summary_test, global_step = epoch)
        
    test_writer.close()

Epoch: 0001 Avg Loss: 0.136
Epoch: 0002 Avg Loss: 0.044
Epoch: 0003 Avg Loss: 0.031
Epoch: 0004 Avg Loss: 0.026
Epoch: 0005 Avg Loss: 0.021
Epoch: 0006 Avg Loss: 0.017
Epoch: 0007 Avg Loss: 0.012
Epoch: 0008 Avg Loss: 0.010
Epoch: 0009 Avg Loss: 0.013
Epoch: 0010 Avg Loss: 0.010
Epoch: 0011 Avg Loss: 0.011
Epoch: 0012 Avg Loss: 0.007
Epoch: 0013 Avg Loss: 0.007
Epoch: 0014 Avg Loss: 0.006
Epoch: 0015 Avg Loss: 0.008
Epoch: 0016 Avg Loss: 0.007
Epoch: 0017 Avg Loss: 0.004
Epoch: 0018 Avg Loss: 0.004
Epoch: 0019 Avg Loss: 0.006
Epoch: 0020 Avg Loss: 0.005
Epoch: 0021 Avg Loss: 0.003
Epoch: 0022 Avg Loss: 0.007
Epoch: 0023 Avg Loss: 0.003
Epoch: 0024 Avg Loss: 0.003
Epoch: 0025 Avg Loss: 0.004
Epoch: 0026 Avg Loss: 0.001
Epoch: 0027 Avg Loss: 0.001
Epoch: 0028 Avg Loss: 0.006
Epoch: 0029 Avg Loss: 0.003
Epoch: 0030 Avg Loss: 0.003
